# Introduction to the Integrated Deepfake Detection System

## Overview

Welcome to the Integrated Deepfake Detection System! This project is a comprehensive effort to tackle the growing problem of deepfake videos using advanced machine learning techniques. As deepfake technology continues to evolve, it poses significant challenges to privacy, security, and authenticity in media. Our system is designed to detect deepfakes by analyzing multiple aspects of video content, specifically focusing on spatial, temporal, and micro-expression features. By integrating these different feature types, we aim to create a robust and reliable detection mechanism capable of identifying manipulated video content.

## Objectives

The primary objective of this project is to develop a deepfake detection system that can accurately distinguish between genuine and manipulated videos. This involves:

1. **Spatial Feature Extraction**: Analyzing individual frames of a video to capture static facial features. This is accomplished using pre-trained Convolutional Neural Networks (CNNs) such as ResNet50 and VGG16, which are fine-tuned for this task.
   
2. **Temporal Feature Extraction**: Understanding how facial features change over time by analyzing sequences of frames. Bidirectional Long Short-Term Memory (BiLSTM) networks are employed to capture temporal dependencies and detect inconsistencies in the flow of facial expressions.

3. **Micro-Expression Analysis**: Focusing on subtle facial movements that are difficult to replicate in deepfake videos. This module uses specialized CNN architectures to extract and analyze micro-expressions, providing an additional layer of detection.

4. **Feature Fusion**: Combining spatial, temporal, and micro-expression features using attention mechanisms to form a comprehensive feature set that enhances detection accuracy. This fusion approach leverages the strengths of each feature type to make a final decision about the authenticity of the video.

## Why This Approach?

Deepfake detection is a challenging task due to the sophistication of the algorithms used to create these fakes. Traditional detection methods that rely on a single type of feature often fail to capture the complexity of manipulations. By integrating spatial, temporal, and micro-expression features, our approach provides a multi-dimensional analysis that significantly improves the likelihood of detecting deepfakes. This holistic strategy addresses the limitations of existing methods and provides a more reliable solution.

### Key Challenges Addressed:

- **Variability in Deepfake Techniques**: Different deepfake algorithms have varying strengths and weaknesses. By analyzing multiple aspects of the video, our system can detect a wide range of manipulations.
- **Subtle Manipulations**: Some deepfakes are so well-crafted that the manipulations are not immediately noticeable to the human eye. Micro-expression analysis helps in detecting these subtle manipulations.
- **Generalization**: Ensuring that the model is not overfitting to specific datasets or types of deepfakes. The fusion of different feature types helps the model generalize better across diverse datasets.

## Dataset

For this project, we use the **FaceForensics++** dataset, which is a benchmark dataset commonly used in deepfake detection research. It contains a collection of both original and manipulated video sequences, providing a diverse set of examples for training and testing the system. The dataset is divided into two main categories:

- **Original Sequences**: Videos that have not been altered, serving as ground truth for authenticity.
- **Manipulated Sequences**: Videos that have been altered using various deepfake techniques, providing examples of fake content.

## Notebook Structure

This Jupyter Notebook is structured to guide you through the different stages of the deepfake detection process:

1. **Data Preprocessing**: Preparing the video frames for feature extraction, including face detection, alignment, and normalization.
   
2. **Model Architecture**: Detailed implementation of the spatial, temporal, and micro-expression feature extraction models. This section includes building and training the CNN (ResNet50, VGG16), BiLSTM, and micro-expression analysis models.

3. **Feature Fusion and Classification**: Combining the extracted features and using attention mechanisms to improve the detection accuracy. The final output layer provides the classification result, indicating whether the video is genuine or a deepfake.

4. **Evaluation and Results**: Testing the trained model on a set of test videos and evaluating its performance using metrics such as accuracy, precision, recall, and F1-score.

5. **Conclusion and Future Work**: Summarizing the findings and discussing potential improvements and future directions for research in deepfake detection.

## Getting Started

To run this notebook, ensure that you have all the necessary dependencies installed. The required libraries are listed in the `requirements.txt` file. Use the following command to install them:

```bash
pip install -r requirements.txt
```

Additionally, make sure you have the FaceForensics++ dataset downloaded and placed in the appropriate directory as specified in the notebook.

## Conclusion

The Integrated Deepfake Detection System represents a significant step towards addressing the challenges posed by deepfake technology. By leveraging multiple feature extraction methods and incorporating attention mechanisms, this system aims to provide a robust solution for identifying manipulated video content. We hope that this project will contribute to the ongoing efforts to maintain the integrity and authenticity of digital media.

Let's get started and dive into the world of deepfake detection!

## **Data Loading**

## **Preprocessing Phase**

## **Model Building**

### 1. Spatial Feature Extractor

In [1]:
import numpy as np
import cv2
import os
import tensorflow as tf
from keras.api.layers import TimeDistributed,Input, Flatten
from keras.api.applications.resnet50 import  ResNet50, preprocess_input as resnet_preprocess
from keras.api.models import  Model
import matplotlib.pyplot as plt

In [3]:
base_model = ResNet50(include_top=False, weights='imagenet', pooling='avg', input_shape=(224,224,3))

In [4]:
model =  Model(inputs=base_model.input, outputs=base_model.output)

In [5]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 230, 230,  │          0 │ input_layer[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 56, 56,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 56, 56,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 56, 56,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 56, 56,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_3_c

 Total params: 23,587,712 (89.98 MB)

 Trainable params: 23,534,592 (89.78 MB)

 Non-trainable params: 53,120 (207.50 KB)

In [6]:
def resnet_preprocess(image_path, target_size=(224,224)):

    img = tf.io.read_file(image_path)
    img = tf.image.decode_image(img, channels=3)
    img = tf.image.resize(img, target_size)

    img = img/255.0
    img = (img-0.5)*2.0

    img_arr = tf.expand_dims(img, axis=0)

    return img_arr


In [7]:
preprocessable_files = []
for filename in os.listdir('../frames'):
    filepath = os.path.join('../frames',filename)
    preprocessable_files.append(filepath)

In [8]:
preprocessed_images = []
for item in preprocessable_files:
    preprocessed_img = resnet_preprocess(item)
    preprocessed_images.append(preprocessed_img)
preprocessed_images_batch = tf.concat(preprocessed_images, axis=0)

In [9]:
len(preprocessed_images_batch)

146

In [10]:
preprocessed_images = np.array(preprocessed_images_batch)

In [11]:
features = model.predict(preprocessed_images)

5/5 ━━━━━━━━━━━━━━━━━━━━ 8s 1s/step


In [12]:
preprocessed_images.shape

(146, 224, 224, 3)

In [13]:
preprocessed_images_expanded = np.expand_dims(preprocessed_images, axis=0)

In [14]:
preprocessed_images_expanded.shape

(1, 146, 224, 224, 3)

In [15]:
features.shape

(146, 2048)

In [16]:
print(features[0])

[0.         0.         0.00772802 ... 4.02363    0.         0.        ]


### 2. Temporal Feature Extractor

In [17]:
from tensorflow import keras
from keras.layers import Bidirectional,LSTM,Input
from keras.models import Model

In [18]:
def build_temporal_feature_extractor(sequence_length, feature_shape):
    input_seq = Input((sequence_length, feature_shape))

    lstm_1 = LSTM(128, return_sequences=True, dropout=0.2)
    lstm_2 = LSTM(64,return_sequences=True, dropout=0.2)

    lstm_out = Bidirectional(lstm_1)(input_seq)
    lstm_out = Bidirectional(lstm_2)(lstm_out)

    model = Model(inputs=input_seq, outputs=lstm_out)
    return model

In [19]:
temporal_feature_extractor = build_temporal_feature_extractor(sequence_length=features.shape[0], feature_shape=features.shape[1])

In [20]:
temporal_feature_extractor.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 146, 2048)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 146, 256)       │     2,229,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 146, 128)       │       164,352 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,393,600 (9.13 MB)

 Trainable params: 2,393,600 (9.13 MB)

 Non-trainable params: 0 (0.00 B)

In [21]:
features.shape

(146, 2048)

In [22]:
features_temp = np.expand_dims(features, axis=0)

In [23]:
features_temp.shape

(1, 146, 2048)

In [24]:
temp_features_extracted = temporal_feature_extractor.predict(features_temp)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 857ms/step


In [25]:
temp_features_extracted

array([[[ 0.01633827, -0.08431578, -0.05848214, ..., -0.01338929,
         -0.26621687, -0.38365075],
        [ 0.04798306, -0.1669606 , -0.0987885 , ..., -0.01242414,
         -0.26590902, -0.41126913],
        [ 0.07777826, -0.2388485 , -0.12598564, ..., -0.01271334,
         -0.26623812, -0.4290023 ],
        ...,
        [ 0.10442451, -0.37411517, -0.2664882 , ..., -0.0488965 ,
         -0.16096166, -0.1845035 ],
        [ 0.11153311, -0.35344535, -0.26045603, ..., -0.03508172,
         -0.11838573, -0.12931089],
        [ 0.12679186, -0.32982087, -0.24678166, ..., -0.01800408,
         -0.06243165, -0.06714023]]], dtype=float32)